In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [ ]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .4
l = .3
mu = .2
g = 10.
h = .05

In [ ]:
# symbolic state
xb, yb, tb = sp.symbols('xb yb tb') # position of the ball
xf, yf = sp.symbols('xf yf') # position of the floor
xdb, ydb, tdb = sp.symbols('xdb ydb tdb') # velocity of the ball
xdf, ydf = sp.symbols('xdf ydf') # velocity of the floor
x = sp.Matrix([
    xb, yb, tb,
    xf, yf,
    xdb, ydb, tdb,
    xdf, ydf
])

# symbolic input
xd2f, yd2f = sp.symbols('xd2f yd2f') # acceleration of the floor
u = sp.Matrix([
    xd2f, yd2f
])

# contact forces
ftf, fnf = sp.symbols('ftf fnf') # floor force
ftc, fnc = sp.symbols('ftc fnc') # ceiling force

In [ ]:
# ball velocity update
xdb_next = xdb + h*ftf/m - h*ftc/m
ydb_next = ydb + h*fnf/m - h*fnc/m - h*g
tdb_next = tdb + r*h*ftf/I + r*h*ftc/I

# ball position update
xb_next = xb + h*xdb_next
yb_next = yb + h*ydb_next
tb_next = tb + h*tdb_next

# floor velocity update
xdf_next = xdf + h*xd2f
ydf_next = ydf + h*yd2f

# floor position update
xf_next = xf + h*xdf_next
yf_next = yf + h*ydf_next

# state update
x_next = sp.Matrix([
    xb_next, yb_next, tb_next,
    xf_next, yf_next,
    xdb_next, ydb_next, tdb_next,
    xdf_next, ydf_next
])

In [ ]:
# relative tangential velocity
sliding_velocity_floor = xdb_next + r*tdb_next - xdf_next
sliding_velocity_ceiling = xdb_next - r*tdb_next

# gap function floor
gap_floor = yb_next - yf_next

# gap function ceiling
gap_ceiling = d - 2.*r - yb_next

# ball distance to boundaries
ball_on_floor = sp.Matrix([
    xb_next - xf_next - l,
    xf_next - xb_next - l
])
ball_on_ceiling = sp.Matrix([
    xb_next - l,
    - xb_next - l
])

In [ ]:
# state bounds
x_max = np.array([
    l, d-2.*r, 1.2*np.pi, # ball config
    l, l/2.,            # floor config
    2., 2., 10.,     # ball vel
    2., 2.          # floor vel
])
# x_min = -  np.array([
#     l, 0., 0., # ball config
#     l, 0.,            # floor config
#     2., 2., 10.,     # ball vel
#     2., 2.          # floor vel
# ])
x_min = - x_max

# input bounds
u_max = np.array([
    30., 30.,          # floor acc
])
u_min = - u_max

# domain bounds
xu = x.col_join(u)
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

In [ ]:
# discrete time dynamics in mode 1
# (ball in the air)

# set forces to zero
f_m1 = {ftf: 0., fnf: 0., ftc: 0., fnc: 0.}

# get dynamics
x_next_m1 = x_next.subs(f_m1)
S1 = AffineSystem.from_symbolic(x, u, x_next_m1)

# build domain
D1 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor and ceiling
gap_floor_m1 = gap_floor.subs(f_m1)
gap_ceiling_m1 = gap_ceiling.subs(f_m1)
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# check domain
assert D1.bounded
assert not D1.empty

In [ ]:
# discrete time dynamics in mode 2
# (ball sticking with the floor, not in contact with the ceiling)

# enforce sticking
fc_m2 = {ftc: 0., fnc: 0.}
ftf_m2 = sp.solve(sp.Eq(sliding_velocity_floor.subs(fc_m2), 0), ftf)[0]
fnf_m2 = sp.solve(sp.Eq(gap_floor.subs(fc_m2), 0), fnf)[0]
f_m2 = fc_m2.copy()
f_m2.update({ftf: ftf_m2, fnf: fnf_m2})

# get dynamics
x_next_m2 = x_next.subs(f_m2)
S2 = AffineSystem.from_symbolic(x, u, x_next_m2)

# build domain
D2 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D2.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D2.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D2.add_symbolic_inequality(xu, ball_on_floor.subs(f_m2))

# friction cone
D2.add_symbolic_inequality(xu, sp.Matrix([ftf_m2 - mu*fnf_m2]))
D2.add_symbolic_inequality(xu, sp.Matrix([- ftf_m2 - mu*fnf_m2]))

# check domain
assert D2.bounded
assert not D2.empty

In [ ]:
# discrete time dynamics in mode 3
# (ball sliding right on the floor, not in contact with the ceiling)

# enforce sticking
f_m3 = {ftf: -mu*fnf_m2, fnf: fnf_m2, ftc: 0., fnc: 0.}

# get dynamics
x_next_m3 = x_next.subs(f_m3)
S3 = AffineSystem.from_symbolic(x, u, x_next_m3)

# build domain
D3 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D3.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D3.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D3.add_symbolic_inequality(xu, ball_on_floor.subs(f_m3))

# positive relative velocity
D3.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_floor.subs(f_m3)]))

# check domain
assert D3.bounded
assert not D3.empty

In [ ]:
# discrete time dynamics in mode 4
# (ball sliding left on the floor, not in contact with the ceiling)

# enforce sticking
f_m4 = {ftf: mu*fnf_m2, fnf: fnf_m2, ftc: 0., fnc: 0.}

# get dynamics
x_next_m4 = x_next.subs(f_m4)
S4 = AffineSystem.from_symbolic(x, u, x_next_m4)

# build domain
D4 = Polyhedron.from_bounds(xu_min, xu_max)

# gap <= 0 with floor
D4.add_symbolic_inequality(xu, sp.Matrix([gap_floor_m1]))

# - gap <= 0 with ceiling
D4.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_m1]))

# ball not falling down the floor
D4.add_symbolic_inequality(xu, ball_on_floor.subs(f_m4))

# negative relative velocity
D4.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_floor.subs(f_m4)]))

# check domain
assert D4.bounded
assert not D4.empty

In [ ]:
# discrete time dynamics in mode 5
# (ball sticking on the ceiling, not in contact with the floor)

# enforce sticking
ff_m5 = {ftf: 0., fnf: 0.}
ftc_m5 = sp.solve(sp.Eq(sliding_velocity_ceiling.subs(ff_m5), 0), ftc)[0]
fnc_m5 = sp.solve(sp.Eq(gap_ceiling.subs(ff_m5), 0), fnc)[0]
f_m5 = ff_m5.copy()
f_m5.update({ftc: ftc_m5, fnc: fnc_m5})

# get dynamics
x_next_m5 = x_next.subs(f_m5)
S5 = AffineSystem.from_symbolic(x, u, x_next_m5)

# build domain
D5 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D5.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D5.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D5.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m5))

# friction cone
D5.add_symbolic_inequality(xu, sp.Matrix([ftc_m5 - mu*fnc_m5]))
D5.add_symbolic_inequality(xu, sp.Matrix([- ftc_m5 - mu*fnc_m5]))

# check domain
assert D5.bounded
assert not D5.empty

In [ ]:
# discrete time dynamics in mode 6
# (ball sliding right on the ceiling, not in contact with the floor)


# enforce sticking
f_m6 = {ftc: -mu*fnc_m5, fnc: fnc_m5, ftf: 0., fnf: 0.}

# get dynamics
x_next_m6 = x_next.subs(f_m6)
S6 = AffineSystem.from_symbolic(x, u, x_next_m6)

# build domain
D6 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D6.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D6.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D6.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m6))

# positive relative velocity
D6.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_ceiling.subs(f_m6)]))

# check domain
assert D6.bounded
assert not D6.empty

In [ ]:
# discrete time dynamics in mode 7
# (ball sliding left on the ceiling, not in contact with the floor)

# enforce sticking
f_m7 = {ftc: mu*fnc_m5, fnc: fnc_m5, ftf: 0., fnf: 0.}

# get dynamics
x_next_m7 = x_next.subs(f_m7)
S7 = AffineSystem.from_symbolic(x, u, x_next_m7)

# build domain
D7 = Polyhedron.from_bounds(xu_min, xu_max)

# - gap <= 0 with floor
D7.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_m1]))

# gap <= 0 with ceiling
D7.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_m1]))

# ball in contact with the ceiling
D7.add_symbolic_inequality(xu, ball_on_ceiling.subs(f_m7))

# negative relative velocity
D7.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_ceiling.subs(f_m7)]))

# check domain
assert D7.bounded
assert not D7.empty

In [ ]:
# list of dynamics
S_list = [S1, S2, S3, S4, S5, S6, S7]

# list of domains
D_list = [D1, D2, D3, D4, D5, D6, D7]

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

In [ ]:
# controller parameters
N = 20
Q = np.diag([
    1., .0, .0,
    1., 1.,
    1., 1., .01,
    1., 1.
])*h
R = np.diag([
    .01, .001
])*h
P = np.zeros((S.nx, S.nx))

# terminal set and cost
X_N = Polyhedron.from_bounds(*[np.zeros(S.nx)]*2)
# X_N = Polyhedron.from_bounds(x_min, x_max)

In [ ]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull, lifted constraints')
# controller.add_reachability_constraints(5)
# controller.prog.setParam('Heuristics', 0)

In [ ]:
# initial condition
x0 = np.array([
    0., 0., np.pi,
    0., 0.,
    0., 0., 0.,
    0., 0.
])
x0 = np.array([
    0., 0.2, 0.,
    0., 0.,
    0., 0., 0.,
    0., 0.
])

In [ ]:
controller.solve_relaxation(x0, {})[1]

In [ ]:
# controller.prog.setParam('MIPFocus', 2)
u_opt, x_opt, ms_opt, cost_opt = controller.feedforward(x0)

from pympc.control.hybrid_benchmark.branch_and_bound import branch_and_bound, best_first, depth_first
def solver(identifier, objective_cutoff):
    return controller.solve_relaxation(x0, identifier, objective_cutoff)
lb = {}
ub = {}
sol = {}
methods = [
    'Convex hull, lifted constraints',
    'Convex hull',
    'Big-M',
    'Traditional formulation'
]
for method in methods:
    print method
    controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method)
    sol_i, lb_i, ub_i = branch_and_bound(solver, depth_first, controller.explore_in_chronological_order)
    lb[method] = lb_i
    ub[method] = ub_i
    sol[method] = sol_i

In [ ]:
for i in range(S.nx):
    print 'x_'+str(i), sum(xt[i]*Q[i,i]*xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u_'+str(i), sum(ut[i]*R[i,i]*ut[i] for ut in u_opt)

In [ ]:
for i in range(S.nx):
    print 'x max_'+str(i), x_max[i], max(xt[i] for xt in x_opt)
    print 'x min_'+str(i), x_min[i], min(xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u max_'+str(i), u_max[i], max(ut[i] for ut in u_opt)
    print 'u min_'+str(i), u_min[i], min(ut[i] for ut in u_opt)

In [ ]:
plot_input_sequence(u_opt, h, (u_min, u_max))

In [ ]:
plot_state_trajectory(x_opt, h, (x_min, x_max))

for i in range(S.nx):
    print i, "min", min(xt[i] for xt in x)
    print i, "max", max(xt[i] for xt in x)
print x_min
print x_max

for i in range(S.nx):
    print sum(xt[i]*Q[i,i]*xt[i] for xt in x)
for i in range(S.nu):
    print sum(ut[i]*R[i,i]*ut[i] for ut in u)

In [ ]:
# x0 = np.array([
#     0., 0.2, 0.,
#     0., 0.,
#     0., 0., -3.,
#     0., 0.
# ])
u_sim = [np.zeros(S.nu)]*50
x_sim, ms_sim = S.simulate(x0, u_opt)
print ms_sim

# Animation

In [ ]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

In [ ]:
tickness = .05
depth = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['ball'].set_object(
    Sphere(r),
    MeshLambertMaterial(color=blue)
)
vis['floor'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ceiling'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ball_orientation'].set_object(
    Cylinder(r/10., r),
    MeshLambertMaterial(color=green)
)

In [ ]:
anim = Animation()
for t, xt in enumerate(x_opt):
    with anim.at_frame(vis, t*h*300) as frame:
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r])
        )
        frame['floor'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-tickness/2.])
        )
        frame['ceiling'].set_transform(
            tf.translation_matrix([0, 0, d+tickness/2.])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)